In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
import numpy as np
import qsharp

INFO:qsharp.clients.iqsharp:Starting IQ# kernel...
DEBUG:qsharp.clients.iqsharp:sending:
%version
INFO:qsharp.clients.iqsharp:Exception while checking if IQ# is ready.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\qsharp\clients\iqsharp.py", line 70, in is_ready
    result = self.component_versions(timeout=6)
  File "C:\ProgramData\Anaconda3\lib\site-packages\qsharp\clients\iqsharp.py", line 128, in component_versions
    self.execute("%version", output_hook=capture, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\qsharp\clients\iqsharp.py", line 152, in execute
    reply = self.kernel_client.execute_interactive(input, output_hook=_output_hook, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\jupyter_client\blocking\client.py", line 325, in execute_interactive
    raise TimeoutError("Timeout waiting for output")
TimeoutError: Timeout waiting for output


Preparing Q# environment...


DEBUG:qsharp.clients.iqsharp:sending:
%version
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': 'ba979e2d-046b-4fa8-a4e9-bee4f7d0a33b', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 11, 7, 878551, tzinfo=datetime.timezone.utc)}, 'msg_id': 'ba979e2d-046b-4fa8-a4e9-bee4f7d0a33b', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': '2d4a2f3d-1879da0b678fba688d61de36', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'ok', 'execution_count': 2, 'payload': [], 'user_expressions': None}, 'buffers': []}
INFO:qsharp.clients.iqsharp:Q# version
{'iqsharp': LooseVersion ('0.6.1904.2013'), 'Jupyter Core': LooseVersion ('1.1.13141.0')}


In [3]:
qsharp.component_versions()

DEBUG:qsharp.clients.iqsharp:sending:
%version
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': '21fda82f-e7d1-418f-babb-0aa024521418', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 11, 20, 686003, tzinfo=datetime.timezone.utc)}, 'msg_id': '21fda82f-e7d1-418f-babb-0aa024521418', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': 'd4cb69ed-29d54f231c07a15a0e3e3e1e', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'ok', 'execution_count': 3, 'payload': [], 'user_expressions': None}, 'buffers': []}


{'iqsharp': LooseVersion ('0.6.1904.2013'),
 'Jupyter Core': LooseVersion ('1.1.13141.0'),
 'qsharp': LooseVersion ('0.5.1903.2902')}

In [4]:
import random

In [5]:
qsharp.get_available_operations_by_namespace()

DEBUG:qsharp.clients.iqsharp:sending:
%who
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': '8b774c7f-00e0-4541-b1f0-bbce0251eb5f', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 11, 28, 759800, tzinfo=datetime.timezone.utc)}, 'msg_id': '8b774c7f-00e0-4541-b1f0-bbce0251eb5f', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': 'b52b635d-a987b016ee423633e3fdf7fb', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'ok', 'execution_count': 4, 'payload': [], 'user_expressions': None}, 'buffers': []}


{'QuantumPerceptron': ['Classify',
  'EncodeDataInQubits',
  'EstimateQuantumClassifierSuccessRate',
  'Validate']}

In [6]:
qsharp.reload()

DEBUG:qsharp.clients.iqsharp:sending:
%workspace reload
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': 'ddf11a78-0bbc-43f9-a2ee-cf4c791979ca', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 11, 45, 887031, tzinfo=datetime.timezone.utc)}, 'msg_id': 'ddf11a78-0bbc-43f9-a2ee-cf4c791979ca', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': 'f6b4e076-b3bc15d04e593610490e0555', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'ok', 'execution_count': 5, 'payload': [], 'user_expressions': None}, 'buffers': []}


In [7]:
def GenerateSplitAngleDataset(n_data_points, seperation_angle, separation_margin):

    sampling_ranges = (
        seperation_angle + separation_margin * np.array([[1, -1], [-1, 1]]) / 2 + np.pi * np.array([[0, 1], [0, 1]])
    ) % (2 * np.pi)

    labels = [random.randint(0, 1) for _ in range(n_data_points)]
    values = [random.uniform(*sampling_ranges[label]) for label in labels]
    
    return values, labels

In [8]:
values, labels = GenerateSplitAngleDataset(10, 0, 0.2)

In [9]:
from QuantumPerceptron import EstimateQuantumClassifierSuccessRate

DEBUG:qsharp.clients.iqsharp:sending:
%who
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': '3d6de369-56de-4066-bd02-6c503c5ba563', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 11, 46, 16824, tzinfo=datetime.timezone.utc)}, 'msg_id': '3d6de369-56de-4066-bd02-6c503c5ba563', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': '67d64911-9bdb36a10a8561a85c9fcd22', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'ok', 'execution_count': 6, 'payload': [], 'user_expressions': None}, 'buffers': []}
DEBUG:qsharp.loader:Trying to load QuantumPerceptron as a Q# namespace.
DEBUG:qsharp.clients.iqsharp:sending:
%who
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': '0a868eb4-5518-401d-a069-e078bafd560a', 'username': None, 'session': '9ac19391-3163e

In [10]:
EstimateQuantumClassifierSuccessRate.simulate(alpha=0, dataPoints=values, labels=labels, nSamples=10)

DEBUG:qsharp.clients.iqsharp:sending:
%simulate QuantumPerceptron.EstimateQuantumClassifierSuccessRate {"alpha": 0, "dataPoints": [0.552430027756981, 3.4105185532910265, 4.616409817389522, 5.751681698798789, 1.121334467241166, 5.335323554829022, 1.0218387672775846, 3.5317416703255717, 1.8949354738681112, 2.985360466486521], "labels": [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], "nSamples": 10}
Constructor on type 'System.ValueTuple`4[[System.Double, System.Private.CoreLib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e],[Microsoft.Quantum.Simulation.Core.IQArray`1[[System.Double, System.Private.CoreLib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e]], Microsoft.Quantum.Simulation.Core, Version=0.6.1904.2013, Culture=neutral, PublicKeyToken=40866b40fd95c7f5],[Microsoft.Quantum.Simulation.Core.IQArray`1[[System.Int64, System.Private.CoreLib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e]], Microsoft.Quantum.Simulation.Core, Version=0.6.1904.2

In [11]:
fn1 = qsharp.compile("""
function Fn1(alpha : Double[]) : Unit {
    Message($"{alpha}");
}
""")

DEBUG:qsharp.clients.iqsharp:sending:

function Fn1(alpha : Double[]) : Unit {
    Message($"{alpha}");
}

DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': '6f40c020-af4e-43ae-9808-f07181c326ac', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 12, 4, 534161, tzinfo=datetime.timezone.utc)}, 'msg_id': '6f40c020-af4e-43ae-9808-f07181c326ac', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': 'cc14cce8-4c1860b03a344222c76b118a', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'ok', 'execution_count': 12, 'payload': [], 'user_expressions': None}, 'buffers': []}


In [12]:
fn1.simulate(alpha=[0, 1])

DEBUG:qsharp.clients.iqsharp:sending:
%simulate Fn1 {"alpha": [0, 1]}
Unable to cast object of type 'System.Double[]' to type 'Microsoft.Quantum.Simulation.Core.IQArray`1[System.Double]'.
DEBUG:qsharp.clients.iqsharp:received:
{'header': {'msg_id': 'dce3ee02-2dfc-4e2b-ab16-6b7b0b66060c', 'username': None, 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_reply', 'version': '5.2.0', 'date': datetime.datetime(2019, 4, 21, 21, 12, 8, 650246, tzinfo=datetime.timezone.utc)}, 'msg_id': 'dce3ee02-2dfc-4e2b-ab16-6b7b0b66060c', 'msg_type': 'execute_reply', 'parent_header': {'msg_id': '8532f20a-34dcd9420a4e1261af4daac9', 'username': 'username', 'session': '9ac19391-3163ebe306b8efde50de7bb4', 'msg_type': 'execute_request', 'version': '5.3'}, 'metadata': None, 'content': {'status': 'error', 'execution_count': 13, 'payload': [], 'user_expressions': None}, 'buffers': []}
